# Deterministic Agent Workflow for Financial Services - Loan Application Processing

This notebook demonstrates a complete **deterministic agent pipeline** for automating a loan application decision process proposed by Team 5 (Full Code) as part of the **Oxford Artificial Intelligence Summit 2025: Autonomous AI Agents** capstone project. The workflow runs a fixed sequence of agent tools and combines their results into a final summary, which is then emailed using a Zapier workflow and MCP integration.

A deterministic workflow was chosen for this project to ensure transparency, reproducibility, and ease of auditing in the loan decision process. By following a clearly defined sequence of steps, the workflow guarantees that every application is evaluated consistently according to the same business rules, making it ideal for regulated environments where explainability and compliance are critical.

The two main sections of this notebook are: 
1. **Deterministic Workflow** — simple baseline flow with hardcoded loan application data
2. **Enhanced Flow** — calls the Zapier MCP server for composing and emailing the final recommendation

Each stage includes **code, agent tool calls, and clear markdown explanations** to guide you through the deterministic process.

## Initial Setup

In [1]:
import pip
!pip install -r requirements.txt

## Set up OpenAI API Key and Zapier webhook URL

1. **Get your OpenAI API Key:**
   - Go to [https://platform.openai.com/api-keys](https://platform.openai.com/api-keys).
   - Log in or sign up for an OpenAI account.
   - Click "Create new secret key" and copy the generated key.

2. **Create a Zapier Webhook URL:**
   - Log in to your [Zapier](https://zapier.com/) account.
   - Click "Create Zap".
   - For the trigger, search for and select **Webhooks by Zapier**.
   - Choose the **Catch Hook** event and continue.
   - Copy the custom webhook URL Zapier provides.
   - Add a "send email" action from Gmail.

3. **Create a `.env` file in your project folder with the following content:**
   ```env
   OPENAI_API_KEY=your-openai-key-here
   MCP_SERVER_URL_SSE=https://hooks.zapier.com/hooks/catch/your_zapier_id/your_path/


## Load in dependencies


In [1]:
from dotenv import load_dotenv
load_dotenv(".env", override=True)

# Import required modules and orchestration pipeline
from data_model import LoanApplicationJourney
from agents import RunContextWrapper, function_tool, CodeInterpreterTool, AgentOutputSchema, Agent, Runner

# Import agent objects
from agents.fraud_agent import fraud_agent
from agents.sla_agent import sla_agent
from agents.recommendation_agent import get_recommendation
from agents.report_agent import report_agent
from agents.interest_rate_agent import interest_rate_agent

# Load Zapier objects
from mcp.zappier_channel import ZapierChannel
import os

## Part 1: Deterministic Workflow – Baseline Agent Flow

For the deterministic flow, the steps of the pipeline are:
1. 🛡️ Check for fraud.
2. ⏱️ Do an SLA check.
3. 💸 Interest rate check.
4. 📝 Make a recommendation for the loan application.
5. 📧 Trigger an email workflow to send an email of the decision.

#### 1.1 **Use function tools defined in the **tools/** folder to analyze fraud, SLA and interest rate and provide the Loan Application Journey object as context to be available to the tools**

These function are defined in the following **tools/** files, which you can notice in the panel on the left:
- check_fraud.py
- check_sla.py
- check_acceptability_threshold.py 
- synthesize_summary.py

#### 1.2 **Define fraud_agent, sla_agent, interest_rate_agent and recommendation_agent and provide the PatientJourney object as context to the agents**
The agents are defined in the following Python files in the **agents/** folder:
- fraud_agent.py
- sla_agent.py
- interest_rate_agent.py
- recommendation_agent.py

#### 1.3 **Define the sequence of agents for deterministic flow**

In [2]:
def deterministic_agent_pipeline(test_app):
    """
    Orchestrates the deterministic loan application pipeline:
    1. Runs fraud detection on the application.
    2. Performs SLA compliance check.
    3. Calculates the interest rate.
    4. Generates a recommendation based on the above results.
    """
    # deterministic orchestration of the loan application process
    fraud_result = fraud_agent.tools[0](RunContextWrapper(test_app))
    sla_result = sla_agent.tools[0](RunContextWrapper(test_app))
    interest_rate_result = interest_rate_agent.tools[0](RunContextWrapper(test_app))

    # Get the recommendation based on the results of the previous steps
    recommendation = get_recommendation(fraud_result, sla_result, interest_rate_result)
    print("Recommendation:", recommendation)

    # Prepare the context for the report agent
    final_context = {
        "recommendation": recommendation,
        "fraud_result": fraud_result,
        "sla_result": sla_result,
        "interest_rate_result": interest_rate_result
    }

    # Generate the summary report
    report_result = report_agent.tools[0](RunContextWrapper(final_context))
    print("System Summary Report:\n", report_result)
    return report_result



#### 1.4 **Define Sample Data**
#### In the sample data below, one can notice the "KYC indicator". 

KYC stands for Know Your Customer.It is a process used by financial institutions and other regulated companies to:
- Verify the identity of their clients.
- Understand the nature of the customer’s activities (to ensure they're legitimate).
- Assess the risk of illegal intentions, like money laundering, fraud, or terrorist financing.



In [6]:
# Create a test LoanApplicationJourney instance
test_app = LoanApplicationJourney(#
    application_id="APP-LOCAL-001",
    submitted_time="2025-06-01T09:00:00",
    reviewed_time="2025-06-01T09:30:00",
    approved_time="2025-06-01T10:00:00",
    rejected_time=None,
    processing_steps={"KYC": 72, "CreditCheck": 50, "FinalApproval": 35},
    flagged_for_fraud=False,
    monthly_income=50000,
    monthly_costs=1000,
    requested_amount=25000,
    monthly_debt=400
)

#### 1.5 **Execute deterministic flow by providing sample data** 

In [7]:
report_result = deterministic_agent_pipeline(test_app)
report_result

/Users/dariaz/Desktop/Oxford AI Summit - Full code summit assignment/Oxford-Summit-Loan-Application/tools/check_acceptability_threshold.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  latest_rate = hist['Close'][-1]


Recommendation: **Recommendation: Reject the Loan Application**

The loan application should be rejected due to SLA violations in the KYC process, which exceeded configured limits. Although there are no fraud indicators detected and the interest rate is excellent, compliance with SLA is critical for maintaining operational integrity and risk management.
System Summary Report:
 📋 Loan Application System Summary
    
This report summarizes the analysis of the loan application system.
Average_processing_time: N/A
Average Requested Amount: N/A
Ratio of Costs to Income: N/A
Analysis Summary: N/A

Regards,
Team 5 - Loan Application Agent



'📋 Loan Application System Summary\n    \nThis report summarizes the analysis of the loan application system.\nAverage_processing_time: N/A\nAverage Requested Amount: N/A\nRatio of Costs to Income: N/A\nAnalysis Summary: N/A\n\nRegards,\nTeam 5 - Loan Application Agent\n'

# Part 2: Enhanced Workflow

### 2.2 **Email Report via MCPTool and Zapier Gmail**

This final part demonstrates the **usage of MCP**:
- A report agent is called dynamically to generate a system summary
- The output is sent as an email using `MCPServerSSE` (which calls a Zapier Gmail webhook)

### 2.1.1 **Trend Analysis**

In [8]:
# Step 4: Generate the report
import pandas as pd
pd_text = pd.read_csv("data/loan_applications_augmented_with_passes.csv").iloc[:10]
pd_text

,application_id,submitted_time,reviewed_time,approved_time,rejected_time,processing_steps,flagged_for_fraud,monthly_income,monthly_costs,requested_amount,monthly_debt
0,APP-0001,2025-06-01T09:00:00,2025-06-01T09:56:00,NaN,2025-06-01T10:23:00,"{'KYC': 86, 'CreditCheck': 21, 'FinalApproval'...",False,4897,2094,23371,550
1,APP-0002,2025-06-02T09:00:00,2025-06-02T09:59:00,NaN,2025-06-02T10:38:00,"{'KYC': 49, 'CreditCheck': 32, 'FinalApproval'...",False,6310,3665,57897,1046
2,APP-0003,2025-06-03T09:00:00,2025-06-03T09:53:00,2025-06-03T10:43:00,NaN,"{'KYC': 28, 'CreditCheck': 13, 'FinalApproval'...",False,6640,1440,27989,1377
3,APP-0004,2025-06-04T09:00:00,2025-06-04T09:38:00,NaN,2025-06-04T10:13:00,"{'KYC': 73, 'CreditCheck': 63, 'FinalApproval'...",False,8492,1713,52029,218
4,APP-0005,2025-06-05T09:00:00,2025-06-05T09:20:00,2025-06-05T09:48:00,NaN,"{'KYC': 24, 'CreditCheck': 46, 'FinalApproval'...",False,3641,3016,53156,1091
5,APP-0006,2025-06-06T09:00:00,2025-06-06T09:23:00,NaN,2025-06-06T09:49:00,"{'KYC': 32, 'CreditCheck': 18, 'FinalApproval'...",True,5012,3599,41532,673
6,APP-0007,2025-06-07T09:00:00,2025-06-07T09:36:00,2025-06-07T10:27:00,NaN,"{'KYC': 17, 'CreditCheck': 64, 'FinalApproval'...",False,3063,3169,49732,719
7,APP-0008,2025-06-08T09:00:00,2025-06-08T09:52:00,NaN,2025-06-08T10:37:00,"{'KYC': 33, 'CreditCheck': 67, 'FinalApproval'...",False,5197,1606,35779,1176
8,APP-0009,2025-06-09T09:00:00,2025-06-09T09:31:00,NaN,2025-06-09T10:02:00,"{'KYC': 17, 'CreditCheck': 35, 'FinalApproval'...",True,8573,3820,56584,495
9,APP-0010,2025-06-10T09:00:00,2025-06-10T09:58:00,NaN,2025-06-10T10:28:00,"{'KYC': 15, 'CreditCheck': 21, 'FinalApproval'...",False,3028,3913,33718,624


In [ ]:
from agents import CodeInterpreterTool,AgentOutputSchema, Agent, Runner
from data_model import TrendAnalysisResult


trend_agent = Agent[TrendAnalysisResult](
    name="TrendAnalyzer",
    instructions="""Analyze the loan_applications_augmented_with_passes.csv CSV file. Return:
1. Average processing times per loan application
2. Average requested amount per loan application
3. Ratio of costs to income per loan application
5. Summary of affordability and final decision 
Also generate a bar chart for average processing time in the format {TrendAnalysisResult}""" ,
    model="gpt-4o-mini",
    tools=[
            CodeInterpreterTool(
                tool_config={"type": "code_interpreter", "container": {"type": "auto"}},
            )
        ],
    output_type=AgentOutputSchema(TrendAnalysisResult)
)

TypeError: AgentOutputSchema.__init__() got an unexpected keyword argument 'strict_json_schema'

#### 2.2.1 **A function for synthesizing the report is implemented**

In [21]:
@function_tool()
def synthesize_report(wrapper: RunContextWrapper[dict]) -> str:
    context = wrapper.context
    return f"""📋 Loan Application System Summary
Dear {context.get('to', 'Daria Zahaleanu')},
Recommendation: {context.get('recommendation', 'No recommendation')}
Fraud Result: {context.get('fraud_result')}
SLA Result: {context.get('sla_result')}
Interest Rate Result: {context.get('interest_rate_result')}

Regards,
Team 5 - Loan Application Agent
"""

In [20]:

# Step 4: Generate the report
analysis_result = await Runner.run(trend_agent, input=pd_text.to_string(index=False))
print(analysis_result)
# Step 5: Await entry (equivalent to `async with`)

final_report_result = await Runner.run(report_agent, 
                          input = "generate the report by calling synthesize_report tool first and then send the synthesized data to create the email",
                          context=analysis_result.final_output)
print("📧 Email result:", final_report_result.final_output)



ModelBehaviorError: Invalid JSON when parsing {"response":{"average_processing_time":42.6,"average_requested_amount":43178.7,"ratio_costs_to_income":0.605492817648208,"analysis_summary":{"Rejected":7,"Approved - Affordable":3}}} for TypeAdapter(OutputType); 2 validation errors for OutputType
response.average_processing_time
  Input should be an object [type=dict_type, input_value=42.6, input_type=float]
    For further information visit https://errors.pydantic.dev/2.11/v/dict_type
response.analysis_summary
  Input should be a valid string [type=string_type, input_value={'Rejected': 7, 'Approved - Affordable': 3}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

### 2.2.2 **MCP Implementation**

In [10]:
zapier_url = os.getenv("MCP_SERVER_URL_SSE", "https://hooks.zapier.com/hooks/catch/your_zapier_id/your_path/")

zapier_channel = ZapierChannel(webhook_url=zapier_url)

 # Send the report via Zapier
zapier_channel.send(
        message=final_report_result,
        metadata={"subject": "Loan Application Recommendation", "to": "anadaria.zahaleanu@email.com"}
    )

print("✅ Report sent via Zapier email webhook.")


✅ Report sent via Zapier email webhook.


In [ ]:
##